# 1. Importing libraries

In [53]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

import warnings
warnings.filterwarnings('ignore')

# 2. Loading Dataset

In [3]:
diabetes_data = pd.read_csv('diabetes.csv')
diabetes_data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


# 3. Data Understanding and EDA

In [6]:
diabetes_data.shape

(768, 9)

In [7]:
diabetes_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [9]:
diabetes_data.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


## Check Imbalanced data

In [10]:
diabetes_data['Outcome'].value_counts()

0    500
1    268
Name: Outcome, dtype: int64

In [11]:
268/500  #Not major imbalance. like 1:10 or 1:100

0.536

# 4. Data Transformation and Model Building

In [13]:
X = diabetes_data.drop('Outcome',axis=1)
y=diabetes_data[['Outcome']]

In [22]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled[:3,:]

array([[ 0.63994726,  0.84832379,  0.14964075,  0.90726993, -0.69289057,
         0.20401277,  0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575,  0.53090156, -0.69289057,
        -0.68442195, -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, -1.28821221, -0.69289057,
        -1.10325546,  0.60439732, -0.10558415]])

In [28]:
X_train,X_test,y_train,y_test = train_test_split(X_scaled, y, stratify=y, random_state=10) #75-25% is default split ratio

In [29]:
X_train.shape,y_train.shape

((576, 8), (576, 1))

In [30]:
X_test.shape,y_test.shape

((192, 8), (192, 1))

In [32]:
576/768 , 192/768

(0.75, 0.25)

In [33]:
y_train.value_counts()

Outcome
0          375
1          201
dtype: int64

In [34]:
201/375 #Same proportion maintained by using stratify in train_test_split

0.536

In [35]:
y_test.value_counts()

Outcome
0          125
1           67
dtype: int64

In [36]:
67/125

0.536

In [46]:
bag_model = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(),
    n_estimators=100,
    max_samples=0.8,  #80% of train data taken 
    oob_score=True,   #out-of-bag samples--those datas missed by the estimators are kind of used to get oob_score which is like validation from X_train itself
    random_state=10
)
bag_model.fit(X_train,y_train)
bag_model.oob_score_

0.7378472222222222

In [47]:
bag_model.score(X_test,y_test)

0.765625

## Standalone model

In [55]:
scores=cross_val_score(DecisionTreeClassifier(),X,y,cv=5)
scores.mean()

0.7149732620320856

## Bagging Classifier

In [52]:
scores=cross_val_score(bag_model,X,y,cv=5)
scores.mean()

0.7617774382480265

## Random Forest

In [54]:
scores=cross_val_score(RandomForestClassifier(),X,y,cv=5)
scores.mean()

0.7708853238265002